In [2]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgb
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import xgboost
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
ROOT            = Path("/kaggle/input/home-credit-risk-dataset-corr-dropped-top-2")
SCHEMA_PATH            = Path("/kaggle/input/schema-home-credit-risk-data")

In [4]:
df = pl.read_csv(ROOT / 'model_abt_pl_corr2_drop.csv').to_pandas()
df.shape

(1526659, 455)

In [5]:
df.head()

,case_id,WEEK_NUM,target,decision_month,decision_weekday,assignmentdate_238D,birthdate_574D,contractssum_5085716L,days120_123L,days180_256L,...,std_collater_valueofguarantee_876L,std_pmts_dpd_1073P,std_pmts_dpd_303P,std_pmts_overdue_1140A,std_pmts_overdue_1152A,count_depth2_a2_num_group2,first_empls_economicalst_849M,first_empls_employer_name_740M,last_empls_economicalst_849M,last_empls_employer_name_740M
0,0,0,0,1,4,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1,0,0,1,4,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2,0,0,1,5,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,3,0,0,1,4,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,4,0,1,1,5,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
def set_data_types(df, schema):
    
    for col, dtype in schema.items():
        if dtype == 'category':
            dtype = 'object'
        df[col] = df[col].astype(dtype)
    return df
    

In [7]:
class Model_Utils:
    @staticmethod
    def model_evals(y_true, y_proba, cutoff = 0.5):
        from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, average_precision_score, roc_auc_score
        """
        Returns model evaluation metrics for a binary classification model

        Parameters:
        -----------
            y_true: int (0,1) 
                Actual binary labels

            y_proba: float (between 0 and 1)
                Probability scores output of model 

        Returns:
        --------
            result: dict
                Dictionary of metrics and their results based on the input
                    - event rate (% predicted 1's)
                    - accuracy
                    - roc_auc
                    - pr_auc
                    - recall
                    - precision
                    - f1
                    - lift
        """

        y_pred = (y_proba > cutoff).astype(int)

        event_rate = y_pred.mean()

        accuracy = accuracy_score(y_true, y_pred)

        roc_auc = roc_auc_score(y_true, y_proba)

        pr_auc = average_precision_score(y_true, y_proba)

        recall = recall_score(y_true, y_pred)

        precision = precision_score(y_true, y_pred)

        f1 = f1_score(y_true, y_pred)

        lift = recall / event_rate

        return {'event_rate': event_rate,
                'acc': accuracy, 
                'roc_auc': roc_auc,
                'pr_auc': pr_auc, 
                'recall': recall, 
                'precision': precision, 
                'f1': f1, 
                'lift': lift}
    
    @staticmethod
    def DumbClassifier(y_true):
        import numpy as np
        y_pred = np.zeros(len(y_true))
        return y_pred
    
    @staticmethod
    def RandomChanceClassifier(y_true):
        import numpy as np
        y_pred = Model_Utils.DumbClassifier(y_true)
        event = y_true.sum()
        ind = np.random.randint(0, len(y_true), size=event)
        y_pred[ind] = 1
        return y_pred
    
    @staticmethod
    def save_model_results(results, schema, filepath):
        if not os.path.exists(filepath):
            score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1_score': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
            scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
        else:
            scores = pd.read_csv(filepath)
        row = []
        
        for key in schema.keys():
            row.append(results[key])
        
        scores.loc[len(scores)] = row
        scores.to_csv(filepath, index = False)
        
    @staticmethod
    def LightGBMClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import lightgbm
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
            params = {'random_state': 42
                     ,'objective': 'binary'
                     ,'verbose': -1
                     ,'n_jobs': -1}
        
        scores = {'params': params,
                        'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1_score': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]

            X_train[cat_cols] = X_train[cat_cols].astype("category")
            X_valid[cat_cols] = X_valid[cat_cols].astype("category")

            lgb = lightgbm.LGBMClassifier(**params)
            lgb.fit(X_train, y_train)
            print(lgb.get_params(deep=True))
            print(lgb._other_params)

            y_proba = lgb.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'LightGBM, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
        
        return scores

    
    @staticmethod
    def train_test_split(X, y, test_size = 0.2):
        from sklearn.model_selection import train_test_split
        import polars as pl
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        print(f"X_train: {X_train.shape}")
        print(f"y_train: {y_train.shape}")
        #pl.from_pandas(pd.concat([y_train, X_train], axis = 1)).write_csv('model_abt_train.csv')
        del X_train
        del y_train
        gc.collect()
        
        print(f"X_test: {X_test.shape}")
        print(f"y_test: {y_test.shape}")
        pl.from_pandas(pd.concat([y_test, X_test], axis = 1)).write_csv('model_abt_test.csv')
        del X_test
        del y_test
        gc.collect()

In [8]:
schema = pd.read_csv(SCHEMA_PATH / "data_schema.csv", names = ['Columns', 'dtype'])a
schema = schema[schema['Columns'].isin(df.columns)]
schema = schema.set_index('Columns')['dtype'].to_dict()
set(schema.values())

{'bool', 'category', 'float64', 'int64', 'int8'}

In [9]:
non_feat = ['case_id', 'WEEK_NUM', 'decision_month', 'decision_weekday', 'target']
num_cols = [x for x, dtype in schema.items() if x not in non_feat and dtype in ['int64', 'float64', 'int8']]
cat_cols = [x for x, dtype in schema.items() if x not in non_feat and dtype == 'category']

In [10]:
df = set_data_types(df, schema)

In [11]:
id_val = df[['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday']]
y = df['target']
X = df[num_cols + cat_cols]
del df

#### Split Data into Train & Test
We will get 20% of the data as holdout. We will tune our model on the 80%.

In [12]:
Model_Utils.train_test_split(X, y)

X_train: (1221327, 449)
y_train: (1221327,)
X_test: (305332, 449)
y_test: (305332,)


In [13]:
pl.read_csv('model_abt_test.csv').head()

target,assignmentdate_238D,birthdate_574D,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,firstquarter_103L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtssum_45A,responsedate_1012D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbdtollast24m_4525197P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt_100L,clientscnt_1022L,…,min_status_219L,max_classificationofcontr_13M,max_contractst_545M,max_description_351M,max_purposeofcred_426M,max_purposeofcred_874M,max_subjectrole_182M,max_subjectrole_93M,min_classificationofcontr_13M,min_contractst_545M,min_description_351M,min_purposeofcred_426M,min_purposeofcred_874M,min_subjectrole_182M,min_subjectrole_93M,first_empl_employedtotal_800L,first_empl_industry_691L,first_familystate_447L,first_incometype_1044T,first_sex_738L,first_housetype_905L,max_depth2_collater_typofvalofguarant_298M,max_depth2_collater_typofvalofguarant_407M,max_depth2_collaterals_typeofguarante_359M,max_depth2_collaterals_typeofguarante_669M,max_depth2_subjectroles_name_541M,max_depth2_subjectroles_name_838M,min_depth2_collater_typofvalofguarant_298M,min_depth2_collater_typofvalofguarant_407M,min_depth2_collaterals_typeofguarante_359M,min_depth2_collaterals_typeofguarante_669M,min_depth2_subjectroles_name_541M,min_depth2_subjectroles_name_838M,first_empls_economicalst_849M,first_empls_employer_name_740M,last_empls_economicalst_849M,last_empls_employer_name_740M
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
0,null,null,0.0,0.0,0.0,0.0,1.0,4.0,1.0,1.0,null,null,null,null,14.0,0.0,1.0,0.0,0.0,6747.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,0.0,0.0,0.0,…,"""D""","""ea6782cc""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""ab3c25cf""","""ab3c25cf""","""a55475b1""","""7241344e""","""a55475b1""","""60c73645""","""5065c2b8""","""a55475b1""","""a55475b1""",null,null,null,"""RETIRED_PENSIONER""","""F""","""OWNED""","""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf""","""9a0c095e""","""8fd95e4b""","""3cbe86ba""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
0,null,null,null,0.0,1.0,0.0,5.0,5.0,8.0,5.0,null,null,null,null,14.0,3.0,4.0,null,null,2206.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,0.0,0.0,0.0,…,"""D""","""ea6782cc""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""ab3c25cf""","""ab3c25cf""","""a55475b1""","""7241344e""","""a55475b1""","""60c73645""","""60c73645""","""a55475b1""","""a55475b1""",null,null,"""MARRIED""","""SALARIED_GOVT""","""F""",null,"""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf""","""9a0c095e""","""8fd95e4b""","""3cbe86ba""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
0,null,null,1441527.5,1.0,1.0,0.0,4.0,3.0,4.0,4.0,null,null,null,null,14.0,1.0,2.0,0.0,63936.85,3342.4001,0.0,0.0,0.0,0.0,0.0,0.0,6.0,-5.0,-9.0,4652.8003,null,0.0,3491.2,0.0,0.0,0.0,…,"""D""","""ea6782cc""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""ab3c25cf""","""ab3c25cf""","""01f63ac8""","""7241344e""","""a55475b1""","""60c73645""","""5065c2b8""","""a55475b1""","""a55475b1""",null,null,null,"""RETIRED_PENSIONER""","""M""","""OWNED""","""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf

##### We will define our baseline vs Random Classifier (random chance) and a Dumb Classifier (predict all majority class). Optimized models should be able to beat these 2.


In [16]:
score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}

##### Dumb Classifier


In [17]:
results  = Model_Utils.model_evals(y, Model_Utils.DumbClassifier(y))
print(results)
results['model_name'] = 'Dumb Classifier'
results['model'] = 'Predict All Major'
results['params'] = None
Model_Utils.save_model_results(results, score_schema, "baseline.csv")
del results
gc.collect()

{'event_rate': 0.0, 'acc': 0.9685627242232876, 'roc_auc': 0.5, 'pr_auc': 0.03143727577671242, 'recall': 0.0, 'precision': 0.0, 'f1': 0.0, 'lift': nan}


496

##### Random Chance Classifier

In [18]:
results  = Model_Utils.model_evals(y, Model_Utils.RandomChanceClassifier(y))
print(results)
results['model_name'] = 'Random Chance Classifier'
results['model'] = 'Randomly Predict at Event Rate'
results['params'] = None
Model_Utils.save_model_results(results, score_schema, "baseline.csv")
del results
gc.collect()


{'event_rate': 0.030934871506996652, 'acc': 0.9395994783379916, 'roc_auc': 0.5002184757424011, 'pr_auc': 0.031450762520636714, 'recall': 0.0313580864274701, 'precision': 0.03186736400787685, 'f1': 0.03161067411600382, 'lift': 1.0136808365400105}


0